In [29]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm_notebook as tqdm

def get_player_statistics(event_id="8896868",home=True):
    URL = 'https://api.sofascore.com/api/v1/event/'+str(event_id)+'/lineups'
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    page = requests.get(URL,headers=headers)
    stats = eval(page.content.decode('utf8').replace("'", '').replace("true","True").replace("false","False"))
    
    home_players = [process_player_statictcs(player) for player in stats["home"]["players"]]
    home_players = [x for x in home_players if x!=None]
    away_players = [process_player_statictcs(player) for player in stats["away"]["players"]]
    away_players = [x for x in away_players if x!=None]
    
    
    return home_players+away_players

def process_player_statictcs(player):
    if player["substitute"]==True:
        return None
    data = {}
    data["name"] = player["player"]["name"]
    data["position"] = player["position"]
    data["rating"] = player["statistics"]["rating"]
    
    return data

#2019-2020 23776
#2020-2021 29415
def get_matches(kolo,season_id):
    URL = 'https://api.sofascore.com/api/v1/unique-tournament/17/season/'+str(season_id)+'/events/round/'+str(kolo)
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    page = requests.get(URL,headers=headers)
    rezultati = eval(page.content.decode('utf8').replace("'", '"').replace("true","True").replace("false","False"))
    return rezultati

def process_match(match):
    try:
        data = {}
        data["event_id"] = match["id"]

        data["homeTeam"] = match["homeTeam"]["name"]
        data["awayTeam"] = match["awayTeam"]["name"]

        data["homeGoals"] = match["homeScore"]["current"]
        data["awayGoals"] = match["awayScore"]["current"]

        stats = get_player_statistics(match["id"])
        if stats == None:
            return None
        for p in range(11):
            data["h_player_"+str(p)] = stats[p]["name"]
            data["h_position_"+str(p)] = stats[p]["position"]
            data["h_score_"+str(p)] = stats[p]["rating"]
        for p in range(11,22):
            data["a_player_"+str(p)] = stats[p]["name"]
            data["a_position_"+str(p)] = stats[p]["position"]
            data["a_score_"+str(p)] = stats[p]["rating"]
        return data
    except KeyError:
        return None

cijela_liga = []
#2019-2020 23776
#2020-2021 29415
for i in tqdm(range(38)):
    cijelo_kolo = [process_match(match) for match in get_matches(i+1,23776)["events"]]
    cijelo_kolo = [x for x in cijelo_kolo if x!=None]
    cijela_liga+=cijelo_kolo
for i in tqdm(range(8)):
    cijelo_kolo = [process_match(match) for match in get_matches(i+1,29415)["events"]]
    cijelo_kolo = [x for x in cijelo_kolo if x!=None]
    cijela_liga+=cijelo_kolo

df = pd.DataFrame(cijela_liga)
df.head()

df.to_csv("PL20192020.csv",index=False)

In [31]:
df

,event_id,homeTeam,awayTeam,homeGoals,awayGoals,h_player_0,h_position_0,h_score_0,h_player_1,h_position_1,...,a_score_18,a_player_19,a_position_19,a_score_19,a_player_20,a_position_20,a_score_20,a_player_21,a_position_21,a_score_21
0,8243388,Liverpool,Norwich City,4,1,Alisson,G,6.9,Trent Alexander-Arnold,D,...,6.8,Marco Stiepermann,M,6.4,Todd Cantwell,M,6.2,Teemu Pukki,F,7.3
1,8243390,West Ham United,Manchester City,0,5,Łukasz Fabiański,G,5.8,Ryan Fredericks,D,...,6.5,Riyad Mahrez,F,9.0,Gabriel Jesus,F,7.4,Raheem Sterling,F,9.5
2,8243397,Bournemouth,Sheffield United,1,1,Aaron Ramsdale,G,6.6,Chris Mepham,D,...,6.5,Enda Stevens,M,6.5,Callum Robinson,F,6.6,David McGoldrick,F,6.3
3,8243396,Watford,Brighton & Hove Albion,0,3,Ben Foster,G,5.5,Kiko Femenia,D,...,6.7,Pascal Groß,F,7.2,Glenn Murray,F,6.6,Jürgen Locadia,F,6.8
4,8243395,Burnley,Southampton,3,0,Nick Pope,G,7.6,Matthew Lowton,D,...,6.7,Danny Ings,F,6.5,Nathan Redmond,F,6.8,Che Adams,F,5.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
451,8896847,West Ham United,Fulham,1,0,Łukasz Fabiański,G,7.8,Fabián Balbuena,D,...,6.7,Tom Cairney,M,7.9,Ademola Lookman,M,6.2,Aleksandar Mitrović,F,6.6
452,8896851,West Bromwich Albion,Tottenham,0,1,Sam Johnstone,G,7.0,Darnell Furlong,D,...,6.9,Tanguy Ndombélé,M,6.6,Heung-Min Son,M,6.6,Harry Kane,F,7.9
453,8896870,Leicester City,Wolverhampton,1,0,Kasper Schmeichel,G,7.3,Wesley Fofana,D,...,6.1,Pedro Neto,F,7.0,Daniel Podence,F,6.6,Raúl Jiménez,F,6.7
454,8896883,Manchester City,Liverpool,1,1,Ederson,G,6.6,Kyle Walker,D,...,6.2,Roberto Firmino,M,6.5,Sadio Mané,M,7.2,Mohamed Salah,F,7.0
